# Simple dashboard

[Colab](https://colab.research.google.com/drive/1v3YuLUErLaTE0mqYNokRT5_iY8GFLGNi?usp=sharing)

In [1]:
%%sh
pip install -q dash
pip install -q dash_core_components
pip install -q dash_html_components
pip install -q dash_table

In [2]:
%%sh
# get ngrok
curl -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.1M  100 13.1M    0     0  49.2M      0 --:--:-- --:--:-- --:--:-- 49.0M


In [3]:
%%writefile dash_app.py

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import calendar
import plotly.express as px
import pandas as pd
import plotly.graph_objs as go

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__)#, external_stylesheets=external_stylesheets)

df = pd.read_csv(
    'https://raw.githubusercontent.com/ashishpatel26/Introduction-to-Time-Series-forecasting/master/international-airline-passengers.csv',
    header=None)
df.columns=['date','count']
df['year']=df['date'].str.slice(0,4).astype(int)
df['month']=df['date'].str.slice(5,7).astype(int)



app.layout = html.Div(children=[
    html.H1(children='Simple Dashboard'),

    html.Div(children='''
        International airline passengers
    '''),

    dcc.Slider(
    id='year-slider',
    min=df['year'].min(),
    max=df['year'].max(),
    marks={str(year): str(year) for year in df['year'].unique()},
    value=df['year'].min(),
    included=False
    ),

    dcc.Dropdown(
        id='dropdown-month',
        options=list({'label': calendar.month_name[m], 'value': m} for m in range(1,13))
        ,multi=True
        #,value='All months'
    ),

    dcc.Graph(
        id='bar-graph',
        figure=go.Figure(data=[go.Bar(x = df['date'], y = df['count'])])
    )
  
])

@app.callback(
    Output('bar-graph', 'figure'),
    Input('year-slider', 'value'),
    Input('dropdown-month', 'value'))
def update_figure(selected_year, selected_month):

    if not selected_month or 'All months' in selected_month:
        filtered_df = df[df.year==selected_year]
    else:
      filtered_df = df[(df.year==selected_year)&(df.month.isin(selected_month))]

    fig = go.Figure(data=[go.Bar(x = filtered_df['date'], y = filtered_df['count'])])

    fig.update_layout(transition_duration=500)

    return fig



if __name__ == '__main__':
    app.run_server(debug=True)

Writing dash_app.py


In [7]:
# launch ngrok
get_ipython().system_raw('./ngrok http 8050 &')

In [8]:
%%sh
# get url with ngrok
curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://e79e-35-199-33-160.ngrok.io


In [9]:
!python dash_app.py

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "dash_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
